In [222]:
import numpy as np
from numpy import array
import csv
import math
TRAIN_FILE="mnist_train.csv"
num_layers=2
num_itr=10
neuron_num=784;
result_class=10;
step = 0.5;
bias=0;
model_file="model.csv"


In [155]:
train_data = np.genfromtxt((TRAIN_FILE), delimiter=',');

In [21]:
print(train_data.shape)

(7000, 785)


`The biases are always 1`

In [223]:
def filldefault(file,layers):
    with open(file,'w') as f:
        for layernum in range(0,layers):
            if( layernum < layers -1 ):
                # fill initial defaults
                for i in range(0,neuron_num*neuron_num-1):
                    f.write("0.5,");
                f.write("0.5\n");
            else:
                # fill initial defaults
                for i in range(0,neuron_num*result_class):
                    f.write("0.5,");
                for i in range((neuron_num)*result_class,neuron_num*(neuron_num) -1):
                    f.write("0,")
                f.write("0\n")
        f.close()

The weights are from first class to next one. first val ~= input1 -> hidden(1,1) . second val ~= input2 -> hidden(1,1)
Biases come after all the weights

In [207]:
filldefault("model.csv",num_layers);  ## Reset model to initial

In [62]:
def activation_function(value):  #implement sigmoid for better results
    return 1/(1+math.exp(-value))

In [221]:
def derivative(value):
    return (1-value)*value;

In [283]:
def front_prop(inputs,model,layers):
    input_vals = inputs
    output_vals = np.array([input_vals])
    for layer in range(0,layers):
#         print(layer)
        input_n = len(input_vals)  # number of inputs of this layer
        if(layer == layers - 1):
            output_n = result_class;
        else:
            output_n = neuron_num;  # number of outputs of this layer
        outputs = np.zeros(output_n) # outputs of this particular layer
        for i in range(0,output_n):
            start = i*input_n;
            end = (i+1)*input_n;
            weights = model[layer,start:end];
            net = np.sum(np.multiply(input_vals,weights)) + bias;
            outputs[i] = activation_function(net);
        if(layer < layers -1):
            output_vals = np.vstack([output_vals,np.array([outputs])]);
        input_vals = outputs;
    return (output_vals,outputs); # giving it as tup as final output will have less size and will cause problems.

In [282]:
def delta(outputs,targets,model):
    layers = len(model);
    deltas = model; # make a copy for structure
    final_out = outputs[1]
    prev_layer = np.subtract(final_out,targets)
    hidden_out = outputs[0]
    for i in range(0,layers):
        newprev = np.zeros(neuron_num)
        prev_out=hidden_out[-(i+1),:]; # outputs of layer prev to this 
        if ( i == 0 ):
            this_out=final_out
            for j in range(0,result_class):
                for k in range(0,neuron_num):
                    p = prev_layer[j]*derivative(this_out[j])*prev_out[k];
                    deltas[layers -1,k+neuron_num*j] = p;
                    newprev[k]+=prev_layer[j]*derivative(this_out[j])*model[-1,k+neuron_num*j];
            prev_layer = newprev;
        else:
            this_out = hidden_out[-i,:];
            for j in range(0,neuron_num):
                for k in range(0,neuron_num): # wt for k -> j
                    p = prev_layer[j]*derivative(this_out[j])*prev_out[k];
                    deltas[-(i+1),k+neuron_num*j] = p;
                    newprev[k]+=prev_layer[j]*derivative(this_out[j])*model[-1,k+neuron_num*j];
            prev_layer = newprev;
    return deltas;

In [280]:
def back_prop(outputs,targets,model):
    layers = len(model);
    newmodel = model;
    deltas = delta(outputs,targets,model);
    for i in range (0,layers):
        for j in range(0,len(model[i])):
            newmodel[i,j]  = model[i,j] - step * deltas[i,j]
    return newmodel

In [281]:
def write_back(file,value):
    with open(file,'w') as f:
        for i in range(0,len(value)):
            for j in range(0,len(value[i]) - 1):
                f.write(str(value[i,j])+",")
            f.write(str(value[i,-1])+"\n")
        f.close();

In [198]:
def loss_fn(output,target):
    return 0.5*np.sum(np.power(np.subtract(output,target),2));

In [279]:
def train(itr,layer,model_param,file):
    while(itr > 0):
        print('Iteration '+str(itr))
        for sample in range(0,len(train_data[:,0])):  # for testing only :C
            result = train_data[sample,0].astype(int)
            targets = np.zeros(result_class)
            targets[result] = 1;
            inputs = train_data[sample,1:]
            outputs = front_prop(inputs,model_param,layer)   # outputs is a tuple
            net_error = loss_fn(outputs[1],targets)
            new_model = back_prop(outputs,targets,model_param)
            model_param = new_model
            print('ITR '+str(itr)+', ERROR FOR SAMPLE NO. '+str(sample)+' : ' + str(net_error))
        itr = itr -1;
    print('Writing trained model to file!')
    write_back(file,model_param)

Model file contains all rows having same number of entries, in last row junk entries are fed!

In [263]:
model_param = np.genfromtxt(model_file,delimiter=",");

In [ ]:
train(1,2,model_param,model_file)

Iteration 1
ITR 1, ERROR FOR SAMPLE NO. 0 : 4.0
ITR 1, ERROR FOR SAMPLE NO. 1 : 2.4411746483126104
ITR 1, ERROR FOR SAMPLE NO. 2 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 3 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 4 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 5 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 6 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 7 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 8 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 9 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 10 : 4.999999286620263
ITR 1, ERROR FOR SAMPLE NO. 11 : 2.4411452630772157
ITR 1, ERROR FOR SAMPLE NO. 12 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 13 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 14 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 15 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 16 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 17 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 18 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 19 : 2.441174

ITR 1, ERROR FOR SAMPLE NO. 160 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 161 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 162 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 163 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 164 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 165 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 166 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 167 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 168 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 169 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 170 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 171 : 2.441174648304899
ITR 1, ERROR FOR SAMPLE NO. 172 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 173 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 174 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 175 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 176 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 177 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 178 : 4.000000000000254
ITR 1, ERROR

ITR 1, ERROR FOR SAMPLE NO. 318 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 319 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 320 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 321 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 322 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 323 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 324 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 325 : 2.441174648304899
ITR 1, ERROR FOR SAMPLE NO. 326 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 327 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 328 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 329 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 330 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 331 : 2.441174648304899
ITR 1, ERROR FOR SAMPLE NO. 332 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 333 : 2.441174648333573
ITR 1, ERROR FOR SAMPLE NO. 334 : 4.000000000000254
ITR 1, ERROR FOR SAMPLE NO. 335 : 2.441174648304899
ITR 1, ERROR FOR SAMPLE NO. 336 : 4.000000000000254
ITR 1, ERROR